# Model Training and Evaluation for Climate Downscaling

ClimateLearn provides a variety of baseline models to perform forecasting and [downscaling](https://uaf-snap.org/how-do-we-do-it/downscaling). In this tutorial, we'll see how to train a [ResNet model](https://en.wikipedia.org/wiki/Residual_neural_network) to do both. This tutorial is intended for use in Google Colab. Before starting, ensure that you are on a GPU runtime.

## Google Colab setup
You might need to restart the kernel after installing ClimateLearn so that your Colab environment knows to use the correct package versions.

In [ ]:
!pip install climate-learn

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

## Data preparation
The second cell of this section can be skipped if the data is already downloaded to your Drive. See the "1b-Data Processing" notebook for more details.

In [ ]:
root = "/content/drive/MyDrive/ClimateLearn"
source = "weatherbench"
dataset = "era5"
low_resolution = "5.625"
hi_resolution = "2.8125"
variable = "2m_temperature"
years = range(1979, 2018)

In [ ]:
from climate_learn.data import download

download(root=root, source=source, dataset=dataset, resolution=Low_resolution, variable=variable)
download(root=root, source=source, dataset=dataset, resolution=hi_resolution, variable=variable)

In [ ]:
from climate_learn.data import DataModuleArgs, DataModule
from climate_learn.data.climate_dataset.args import ERA5Args
from climate_learn.data.tasks.args import DownscalingArgs

lowres_data_args = ERA5Args(
    root_dir=f"{root}/data/{source}/{dataset}/{low_resolution}/",
    variables=[variable],
    years=years
)

highres_data_args = ERA5Args(
    root_dir=f"{root}/data/{source}/{dataset}/{hi_resolution}",
    variables=[variable],
    years=years
)

downscaling_args = DownscalingArgs(
    dataset_args=data_args,
    highres_dataset_args=highres_data_args,
    in_vars=[variable],
    out_vars=[variable],
    subsample=6,
)

data_module_args = DataModuleArgs(
    task_args=downscaling_args,
    train_start_year=1979,
    val_start_year=2015,
    test_start_year=2017,
    end_year=2018
)

data_module = DataModule(
    data_module_args=data_module_args,
    batch_size=128,
    num_workers=1
)

## Model initialization

This is almost exactly the same as the previous section, except we specify the task to be "downscaling".

In [ ]:
from climate_learn.models import load_model
from torch.optim import AdamW

model_kwargs = {
    "in_channels": 1,
    "out_channels": 1,
    "n_blocks": 4
}

optim_kwargs = {
    "lr": 1e-4,
    "weight_decay": 1e-5,
    "warmup_epochs": 1,
    "max_epochs": 5,
    "optimizer": AdamW
}

model_module = load_model(
    name="resnet",
    task="downscaling",
    model_kwargs=model_kwargs,
    optim_kwargs=optim_kwargs
)

The note about `in_channels` and `out_channels` from the section in forecasting notebook applies here too.

In [ ]:
from climate_learn.models import set_climatology
set_climatology(model_module, data_module)

## Training

Unlike forecasting, we do not fit a linear regression baseline. This is not currently implemented for the downscaling task.

In [ ]:
from climate_learn.training import Trainer

trainer = Trainer(
    seed=0,
    accelerator="gpu",
    precision=16,
    max_epochs=1
)

trainer.fit(model_module, data_module)

## Evaluation

In [ ]:
trainer.test(model_module, data_module)

Congratulations! Now you know how to train and evaluate baseline models with ClimateLearn. Please visit our [docs](https://climatelearn.readthedocs.io/en/latest/user-guide/models.html) to learn more.